# Lambert Solver Izzo 2015 Algorithm

To do:

- Compute delta V gains
- Idea: compare to Hohmann transfer
- Make GitHub page, including elaborate README.md
- Implement multi-rev Lambert
    


### Import packages

In [2]:
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pykep as pk
import sgp4 as sg
import pandas as pd
from datetime import date, timedelta, datetime
from astroquery.jplhorizons import Horizons
import datetime
from sgp4.api import jday
import sys
from dateutil.relativedelta import relativedelta
import juliandate as julian
from numpy.linalg import inv


### Constants

In [18]:
# NOTE: all constants in SI units

G = 6.6743e-11  # Gravitational constant
AU = 149597870700.0  # Astronomical unit
mu_Sun = 1.32712440018e20  # Gravitational parameter Sun
mu_Earth = 398600441800000.0  # Gravitational parameter Earth
mu_Mars = 4.282837e13  # Gravitationnal parameter Mars
mu_Venus = 3.24859e14 # Gravitational parameter Venus
R_Earth = 6.3781e6 # Earth radius
R_Mars = 3.3895e6 # Mars radius
R_Venus = 6051.8e3 # Venus radius

# Mission geometry input
r_park_Earth = 500e3 + R_Earth  # m, 500km parking orbit Earth assumed
r_park_Mars = 250e3 + R_Mars  # m, 250 km parking orbit Mars
r_flyby_Venus = 307e3 + R_Venus #m, 307 km flyby altitude
r_Mars_Sun = 1.524 * AU
r_Earth_Sun = AU
r_Venus_Sun = 0.72*AU

# Sphere of influence radius of the Earth
r_SI = r_Earth_Sun * (mu_Earth / mu_Sun)**(2/5)

# 3x1 vector magnitude
def magnitude(a):
    return np.sqrt(a[0] ** 2 + a[1] ** 2 + a[2] ** 2)

print(r_SI / 1000)


924646.795104645


## Izzo Code

In [4]:
# This will be used in the compxy(lam, T) function
def hypergeometricF(z, tol):
    Sj = 1
    Cj = 1
    err = 1
    Cj1 = 0
    Sj1 = 0
    j = 0
    while err > tol:
        Cj1 = Cj * (3 + j) * (1 + j) / (2.5 + j) * z / (j + 1)
        Sj1 = Sj + Cj1
        err = np.abs(Cj1)
        Sj = Sj1
        Cj = Cj1
        j += 1

    return Sj


def x2tof2(x, N, lam):
    a = 1 / (1 - x * x)
    if a > 0:  # Ellipse requirement
        alfa = 2 * np.arccos(x)
        beta = 2 * np.arcsin(np.sqrt(lam**2 / a))
        if lam < 0:
            beta = -beta
        tof = (
            a
            * np.sqrt(a)
            * ((alfa - np.sin(alfa)) - (beta - np.sin(beta)) + 2 * np.pi * N)
        ) / 2
        return tof
    else:
        alfa = 2 * np.arccosh(x)
        beta = 2 * np.arcsinh(np.sqrt(-lam * lam / a))
        if lam < 0:
            beta = -beta
        tof = -a * np.sqrt(-a) * ((beta - np.sinh(beta)) - (alfa - np.sinh(alfa))) / 2
        return tof


def x2tof(x, N, lam):
    battin = 0.01
    lagrange = 0.2
    dist = np.abs(x - 1)
    if dist < lagrange and dist > battin:
        tof = x2tof2(x, N, lam)
        return tof

    K = lam * lam
    E = x * x - 1
    rho = np.abs(E)
    z = np.sqrt(1 + K * E)
    if dist < battin:
        eta = z - lam * x
        S1 = 0.5 * (1 - lam - x * eta)
        Q = hypergeometricF(S1, 1e-11)
        Q = 4 / 3 * Q
        tof = (eta**3 * Q + 4 * lam * eta) / 2 + N * np.pi / np.power(rho, 1.5)
        return tof
    else:
        y = np.sqrt(rho)
        g = x * z - lam * E
        d = 0
        if E < 0:
            l = np.arccos(g)
            d = N * np.pi + l
        else:
            f = y * (z - lam * x)
            d = np.log(f + g)
        tof = (x - lam * z - d / y) / E
        return tof

def compxy(lam, T):
    M = 0
    if np.abs(lam) > 1 or T < 0:
        sys.exit()

    # Define maximum number of revolutions

    M_max = np.floor(T / np.pi)

    # Define non-dimensional time for single revolution and x = 0

    T_00 = np.arccos(lam) + lam * np.sqrt(1 - lam**2)

    T_0 = T_00 + M_max * np.pi

    # Define y as function of x
    def y(x):
        return np.sqrt(1 - lam ** (2) * (1 - x**2))

    # First derivative
    def funprime(x):
        dTdx = (
            1 / (1 - x**2) * (3 * x2tof(x, 0, lam) * x - 2 + 2 * lam**3 * x / y(x))
        )
        return dTdx

    # Second derivative
    def funprime2(x):
        d2Tdx2 = (
            1
            / (1 - x**2)
            * (
                3 * x2tof(x, 0, lam)
                + 5 * x * funprime(x)
                + 2 * (1 - lam**2) * lam ** (3) / (y(x) ** (3))
            )
        )
        return d2Tdx2

    def funprime3(x):
        return (
            1
            / (1 - x**2)
            * (
                7 * x * funprime2(x)
                + 8 * funprime(x)
                - 6 * (1 - lam**2) * lam ** (5) * x / (y(x) ** (5))
            )
        )

    if M_max > 0:
        if T < T_0:
            epsilon_H = 1
            T_min = T_0
            it_H = 0
            x_0 = 0
            x_new = 0

            while True:
                # print(str(x_0) + " Halley iteration" + str(it_H))
                if funprime(x_0) != 0:
                    x_new = x_0 - funprime(x_0) * funprime2(x_0) / (
                        funprime2(x_0) * funprime2(x_0)
                        - funprime(x_0) * funprime3(x_0) / 2
                    )
                epsilon_H = np.abs(x_0 - x_new)
                if epsilon_H < 1e-13 or it_H > 12:
                    break
                T_min = x2tof(x_new, M_max, lam)
                x_0 = x_new
                it_H += 1

            if T_min > T:
                M_max -= -1

    # Non-dimensional time definition for x = 1
    T_1 = 2 / 3 * (1 - lam**3)

    # Define x0 initial guess from given T value
    if T >= T_00:
        x_0 = -(T - T_00) / (T - T_00 + 4)
    elif T <= T_1:
        x_0 = (
            5 / 2 * (T_1 * (T_1 - T) / (T * (1 - lam**5))) + 1
        )  # Improved expression
    elif T_1 < T and T < T_0:
        x_0 = np.power((T / T_00), 0.69314718055994529 / np.log(T_1 / T_00)) - 1

    # Perform Householder iteration to find x and y
    # From the paper, for M = 0, the error between xn+1 and xn should be less than e-5

    epsilon = 1
    it = 0
    it_max = 15
    x_new = 0

    while epsilon > 1e-5 and it < it_max:
        # print(str(x_0) + "Iteration first Householder" + str(it))
        tof = x2tof(x_0, 0, lam)
        delta = tof - T
        x_new = x_0 - delta * (funprime(x_0) ** 2 - delta * funprime2(x_0) / 2) / (
            funprime(x_0) * (funprime(x_0) ** 2 - delta * funprime2(x_0))
            + funprime3(x_0) * delta ** (2) / 6
        )
        epsilon = np.abs(x_new - x_0)
        x_0 = x_new
        it += 1

    return x_new, y(x_new)

def LambertIzzo(r_1, r_2, mu, t):
    # Require mu and t to be greater than zero
    if mu < 0 or t < 0:
        print("Error: gravitational parameter and time have to be positive")
        sys.exit()

    # Define magnitudes

    r_1_mag = magnitude(r_1)  # m
    r_2_mag = magnitude(r_2)  # m

    # Define transfer angle theta

    costheta = np.dot(r_1, r_2) / (r_1_mag * r_2_mag)
    theta = np.arccos(costheta)

    # Define c

    c = r_2 - r_1  # Vector
    c_mag = magnitude(c)

    # Define semiperimeter s

    s = (r_1_mag + r_2_mag + c_mag) / 2  # m

    # Define radial unit vectors and perpendicular

    i_r_1 = r_1 / r_1_mag  # Unit vector
    i_r_2 = r_2 / r_2_mag  # Unit vector
    i_h = np.cross(i_r_1, i_r_2)  # Unit vector
    i_h = i_h / magnitude(i_h)

    # Lambda parameter

    lam2 = 1 - c_mag / s
    lam = np.sqrt(lam2)

    if (r_1[0] * r_2[1] - r_1[1] * r_2[0]) < 0:
        lam = -lam
        i_t_1 = np.cross(i_r_1, i_h)
        i_t_2 = np.cross(i_r_2, i_h)

    else:
        i_t_1 = np.cross(i_h, i_r_1)
        i_t_2 = np.cross(i_h, i_r_2)

    # Introduce non-dimensional time of flight
    T = np.sqrt((2 * mu) / (s**3)) * t

    x, y = compxy(lam, T)

    # Auxiliary functions to find delta V values

    gamma = np.sqrt((mu * s) / 2)
    rho = (r_1_mag - r_2_mag) / c_mag
    sigma = np.sqrt(1 - rho**2)

    # TO DO: IMPLEMENT ITERATION OVER X AND Y LISTS, NOW FOR ONE VALUE ONLY

    # Radial and tangential velocity components
    V_r_1 = gamma * ((lam * y - x) - rho * (lam * y + x)) / r_1_mag
    V_r_2 = -gamma * ((lam * y - x) + rho * (lam * y + x)) / r_2_mag
    V_t_1 = gamma * sigma * (y + lam * x) / r_1_mag
    V_t_2 = gamma * sigma * (y + lam * x) / r_2_mag

    # Velocity vectors and their magnitudes
    v_1 = V_r_1 * i_r_1 + V_t_1 * i_t_1
    v_2 = V_r_2 * i_r_2 + V_t_2 * i_t_2

    v_1_mag = magnitude(v_1)
    v_2_mag = magnitude(v_2)

    # print(v_1, v_2)
    # print(v_1_mag, v_2_mag)

    return v_1, v_2, v_1_mag, v_2_mag


## Delta_V calculators

In [5]:
'''
delta_V_Mars_to_Earth: this definition computes the incorrect delta_V values; it is 
assumed that planet velocity and velocity vectors from Lambert's targeter are
aligned, which is not the case for a Lambert problem (only for Hohmann, do not use anymore)
'''

def delta_V_Mars_to_Earth(v_1, v_2):
    # v_1 and #v_2 are defined in the heliocentric frame, vectors!!!
    # The departure at Mars will first be analysed
    # ASSUMPTIONS: - circular parking orbit
    #              - orbital planes equal (no inclination change)
    #              - Sphere of Influence at infitiy
    #              - Patched conics approach

    # Parking orbit velocity at Mars
    v_park_Mars = np.sqrt(mu_Mars / r_park_Mars)  # m/s

    # Mars' velocity in heliocentric frame
    v_Mars = np.sqrt(mu_Sun / r_Mars_Sun)

    # Planetocentric hyperbolic excess velocity at Mars
    v_inf_Mars = np.abs(v_Mars - v_1)

    # Planetocentric periapsis velocity of the hyperbola and departure delta V
    v_p_Mars = np.sqrt(v_inf_Mars**2 + (2 * mu_Mars) / r_park_Mars)
    delta_v_1 = np.abs(v_p_Mars - v_park_Mars)

    # Same procedure for Earth
    v_park_Earth = np.sqrt(mu_Earth / r_park_Earth)
    v_Earth = np.sqrt(mu_Sun / r_Earth_Sun)
    v_inf_Earth = np.abs(v_Earth - v_2)
    v_p_Earth = np.sqrt(v_inf_Earth**2 + 2 * mu_Earth / r_park_Earth)
    delta_v_2 = np.abs(v_p_Earth - v_park_Earth)

    # Total delta V is simply the sum of both components

    delta_v = delta_v_1 + delta_v_2

    return delta_v

'''
Correct functions displayed below
Titles self-explanatory
inputs: jd - julian date of departure
        jd_arrival - julian date of arrival
        v_1 - departure velocity vector from the Izzo algorithm
        v_2 - arrival velocity vector from the Izzo algorithm
        dim - selected dimension, 2 and 3 are possibilities, otherwise error is displayed
'''

def delta_V_Mars_to_Earth_corrected(jd, jd_arrival, v_1, v_2, dim):
    if dim == 2:
        # Parking orbit velocity at Mars
        v_park_Mars = np.sqrt(mu_Mars / r_park_Mars) # m/s

        # Venus and Mars velocities, vectors, in heliocentric frame
        v_Mars, v_Earth = VelVecs(jd, jd_arrival)
        v_Mars[2] = 0
        v_Earth[2] = 0
        v_1[2] = 0
        v_2[2] = 0
        
        # Planetocentric hyperbolic excess velocity at Mars
        v_inf_Mars = v_1 - v_Mars
        v_inf_Mars_mag = magnitude(v_inf_Mars)

        # Planetocentric periapsis velocity of the hyperbola and departure delta V
        v_p_Mars = np.sqrt(v_inf_Mars_mag**2 + (2 * mu_Mars) / r_park_Mars)
        delta_v_1 = np.abs(v_p_Mars - v_park_Mars)

        # Insertion needed at Earth
        # Parking orbit velocity at Earth
        v_park_Earth = np.sqrt(mu_Earth / r_park_Earth) # m/s

        # Planetocentric hyperbolic excess velocity at Earth
        v_inf_Earth = v_2 - v_Earth
        v_inf_Earth_mag = magnitude(v_inf_Earth)

        # Planetocentric periapsis velocity of the hyperbola and arrival delta v
        v_p_Earth = np.sqrt(v_inf_Earth_mag**2 + 2 * mu_Earth / r_park_Earth)
        delta_v_2 = np.abs(v_p_Earth - v_park_Earth)

        return delta_v_1, delta_v_2

    elif dim == 3:
        # Parking orbit velocity at Mars
        v_park_Mars = np.sqrt(mu_Mars / r_park_Mars) # m/s

        # Venus and Mars velocities, vectors, in heliocentric frame
        v_Mars, v_Earth = VelVecs(jd, jd_arrival)
        
        # Planetocentric hyperbolic excess velocity at Mars
        v_inf_Mars = v_1 - v_Mars
        v_inf_Mars_mag = magnitude(v_inf_Mars)

        # Planetocentric periapsis velocity of the hyperbola and departure delta V
        v_p_Mars = np.sqrt(v_inf_Mars_mag**2 + (2 * mu_Mars) / r_park_Mars)
        delta_v_1 = np.abs(v_p_Mars - v_park_Mars)

        # Insertion needed at Earth
        # Parking orbit velocity at Earth
        v_park_Earth = np.sqrt(mu_Earth / r_park_Earth) # m/s

        # Planetocentric hyperbolic excess velocity at Earth
        v_inf_Earth = v_2 - v_Earth
        v_inf_Earth_mag = magnitude(v_inf_Earth)

        # Planetocentric periapsis velocity of the hyperbola and arrival delta v
        v_p_Earth = np.sqrt(v_inf_Earth_mag**2 + 2 * mu_Earth / r_park_Earth)
        delta_v_2 = np.abs(v_p_Earth - v_park_Earth)

        return delta_v_1, delta_v_2

    else:
        return print("ERROR: incorrect dimension selected.")



def delta_V_Mars_to_Venus(jd, jd_arrival, v_1, v_2, dim):

    '''
    In this function, two inputs are required: v1 and v2 as calculated from the Lambert targeter
    The outputs are the velocity increase needed at the departure planet Mars and the PLANETOCENTRIC
    v_infinity experienced upon Venus arrival, just before the gravity assist.

    UPDATE: jd and jd_arrival required as inputs, so that entire function becomes more generic, also dimension added
    '''
    
    if dim == 2:
        # Parking orbit velocity at Mars
        v_park_Mars = np.sqrt(mu_Mars / r_park_Mars)  # m/s

        # Venus and Mars velocities, vectors, in heliocentric frame
        v_Mars, v_Venus = VelVecsVenus(jd, jd_arrival)
        v_Mars[2] = 0
        v_Venus[2] = 0
        
        # Planetocentric hyperbolic excess velocity at Mars
        v_inf_Mars = v_1 - v_Mars
        v_inf_Mars_mag = magnitude(v_inf_Mars)

        # Planetocentric periapsis velocity of the hyperbola and departure delta V
        v_p_Mars = np.sqrt(v_inf_Mars_mag**2 + (2 * mu_Mars) / r_park_Mars)
        delta_v_1 = np.abs(v_p_Mars - v_park_Mars)

        # No insertion needed at Venus, but planetocentric v_infinity should be computed
        v_inf_Venus = v_2 - v_Venus

        return delta_v_1, v_inf_Venus

    elif dim == 3:
            # Parking orbit velocity at Mars
        v_park_Mars = np.sqrt(mu_Mars / r_park_Mars)  # m/s

        # Venus and Mars velocities, vectors, in heliocentric frame
        v_Mars, v_Venus = VelVecsVenus(jd, jd_arrival)
        
        # Planetocentric hyperbolic excess velocity at Mars
        v_inf_Mars = v_1 - v_Mars
        v_inf_Mars_mag = magnitude(v_inf_Mars)

        # Planetocentric periapsis velocity of the hyperbola and departure delta V
        v_p_Mars = np.sqrt(v_inf_Mars_mag**2 + (2 * mu_Mars) / r_park_Mars)
        delta_v_1 = np.abs(v_p_Mars - v_park_Mars)

        # No insertion needed at Venus, but planetocentric v_infinity should be computed
        v_inf_Venus = v_2 - v_Venus

        return delta_v_1, v_inf_Venus
    
    else:
        return print("ERROR: incorrect dimension selected.") 




## Celestial body positions/velocities

In [6]:
# Convert this into a function too, so that it can universally be used within plot generation over time array

"""
PosVecs: This function returns the position of Mars (r_1) and Earth (r_2) in the inertial celestial reference frame,
Sun-centered, at time t1 of departure (input) and t2 of arrival (input), in Julian days. r_1 and r_2 are the centers of
the celestial bodies with respect to the center of the Sun

PosVecsVenus: This function returns the r_1 and r_2 vectors for the gravity assist maneuver about Venus, in the same
frame and with the same input parameters. r_2 is now the location of Venus

PosVecsEarth: This function computes the positional vectors after the gravity assist maneuver, r_1 is now Venus and r_2 Earth

VelVecsVenus: Computes v_1 and v_2, the heliocentric velocities of Mars (1) and Venus (2), from astroquery

VelVecsEarth: Same as Venus, now for Venus and Earth

"""


def PosVecs(jd, jd_arrival):
    # Define Mars epoch, TODAY 00:00:00
    obj_Mars = Horizons(id="499", location="500@10", epochs=jd)

    eph_Mars = obj_Mars.vectors()

    cart_Mars = eph_Mars["x", "y", "z"]

    # Define Earth epoch, ARRIVAL DAY 00:00:00
    obj_Earth = Horizons(id="399", location="500@10", epochs=jd_arrival)

    eph_Earth = obj_Earth.vectors()

    cart_Earth = eph_Earth["x", "y", "z"]

    # Create vectors, r_1 defined at Mars, r_2 defined at Earth

    r_1 = np.array([cart_Mars[0]["x"], cart_Mars[0]["y"], cart_Mars[0]["z"]]) * AU
    r_2 = np.array([cart_Earth[0]["x"], cart_Earth[0]["y"], cart_Earth[0]["z"]]) * AU

    return r_1, r_2

def VelVecs(jd, jd_arrival):
    # Define Mars epoch, TODAY 00:00:00
    obj_Mars = Horizons(id="499", location="500@10", epochs=jd)

    eph_Mars = obj_Mars.vectors()

    cart_Mars = eph_Mars["vx", "vy", "vz"]

    # Define Earth epoch, ARRIVAL DAY 00:00:00
    obj_Earth = Horizons(id="399", location="500@10", epochs=jd_arrival)

    eph_Earth = obj_Earth.vectors()

    cart_Earth = eph_Earth["vx", "vy", "vz"]

    # Create vectors, r_1 defined at Mars, r_2 defined at Earth

    v_1 = np.array([cart_Mars[0]["vx"], cart_Mars[0]["vy"], cart_Mars[0]["vz"]]) * AU / 24 /3600
    v_2 = np.array([cart_Earth[0]["vx"], cart_Earth[0]["vy"], cart_Earth[0]["vz"]]) * AU / 24 / 3600

    return v_1, v_2

def PosVecsVenus(jd, jd_arrival):
    # Define Mars epoch, TODAY 00:00:00
    obj_Mars = Horizons(id="499", location="500@10", epochs=jd)

    eph_Mars = obj_Mars.vectors()

    cart_Mars = eph_Mars["x", "y", "z"]

    # Define Earth epoch, ARRIVAL DAY 00:00:00
    obj_Venus = Horizons(id="299", location="500@10", epochs=jd_arrival)

    eph_Venus = obj_Venus.vectors()

    cart_Venus = eph_Venus["x", "y", "z"]

    # Create vectors, r_1 defined at Mars, r_2 defined at Earth

    r_1 = np.array([cart_Mars[0]["x"], cart_Mars[0]["y"], cart_Mars[0]["z"]]) * AU
    r_2 = np.array([cart_Venus[0]["x"], cart_Venus[0]["y"], cart_Venus[0]["z"]]) * AU

    return r_1, r_2

def PosVecsEarth(jd, jd_arrival):
    # Define Mars epoch, TODAY 00:00:00
    obj_Venus = Horizons(id="299", location="500@10", epochs=jd)

    eph_Venus = obj_Venus.vectors()

    cart_Venus = eph_Venus["x", "y", "z"]

    # Define Earth epoch, ARRIVAL DAY 00:00:00
    obj_Earth = Horizons(id="399", location="500@10", epochs=jd_arrival)

    eph_Earth = obj_Earth.vectors()

    cart_Earth = eph_Earth["x", "y", "z"]

    # Create vectors, r_1 defined at Mars, r_2 defined at Earth

    r_1 = np.array([cart_Venus[0]["x"], cart_Venus[0]["y"], cart_Venus[0]["z"]]) * AU
    r_2 = np.array([cart_Earth[0]["x"], cart_Earth[0]["y"], cart_Earth[0]["z"]]) * AU

    return r_1, r_2

def VelVecsVenus(jd, jd_arrival):
    # Define Mars epoch, TODAY 00:00:00
    obj_Mars = Horizons(id="499", location="500@10", epochs=jd)

    eph_Mars = obj_Mars.vectors()

    cart_Mars = eph_Mars["vx", "vy", "vz"]

    # Define Earth epoch, ARRIVAL DAY 00:00:00
    obj_Venus = Horizons(id="299", location="500@10", epochs=jd_arrival)

    eph_Venus = obj_Venus.vectors()

    cart_Venus = eph_Venus["vx", "vy", "vz"]

    # Create vectors, r_1 defined at Mars, r_2 defined at Earth

    v_1 = np.array([cart_Mars[0]["vx"], cart_Mars[0]["vy"], cart_Mars[0]["vz"]]) * AU / 24 /3600
    v_2 = np.array([cart_Venus[0]["vx"], cart_Venus[0]["vy"], cart_Venus[0]["vz"]]) * AU / 24 / 3600

    return v_1, v_2

def VelVecsEarth(jd, jd_arrival):
    # Define Mars epoch, TODAY 00:00:00
    obj_Venus = Horizons(id="299", location="500@10", epochs=jd)

    eph_Venus = obj_Venus.vectors()

    cart_Venus = eph_Venus["vx", "vy", "vz"]

    # Define Earth epoch, ARRIVAL DAY 00:00:00
    obj_Earth = Horizons(id="399", location="500@10", epochs=jd_arrival)

    eph_Earth = obj_Earth.vectors()

    cart_Earth = eph_Earth["vx", "vy", "vz"]

    # Create vectors, r_1 defined at Mars, r_2 defined at Earth

    v_1 = np.array([cart_Venus[0]["vx"], cart_Venus[0]["vy"], cart_Venus[0]["vz"]]) * AU / 3600 / 24
    v_2 = np.array([cart_Earth[0]["vx"], cart_Earth[0]["vy"], cart_Earth[0]["vz"]]) * AU / 3600 / 24

    return v_1, v_2




## Date array

In [7]:
"""
Return an array of dates to iterate over, for porkchop usage
New function: if simple == 1, only one date per month will be returned
            Also, simplet gives convention [YR, MNTH, DAY]
            Not simple gives other way around (sloppy)
"""

def datesyear(year, simple):
    if simple == 1:
        start_date = datetime.date(2029, 4, 29)
        end_date = datetime.date(2029, 5, 6)
        delta = datetime.timedelta(days=1)

        date_array = []
        current_date = start_date

        while current_date <= end_date:
            date_array.append([current_date.year, current_date.month, current_date.day])
            current_date += delta

        return date_array
    
    elif simple == 2:
        date_array = []
        
        for month in range(1,13):
            start_date = datetime.date(year, month, 1)
            mid_date = datetime.date(year,month,15)
            date_array.append([start_date.year , start_date.month, start_date.day])
            date_array.append([mid_date.year, mid_date.month, mid_date.day])
        
        return date_array
    
    else:
        # Initialize an empty array
        date_array = []

        # Iterate over each month
        for month in range(1, 13):
            # Define the start date as the first day of the specified month
            start_date = datetime.date(year, month, 1)
            mid3_date = datetime.date(year, month, 5)
            mid1_date = datetime.date(year, month, 10)
            mid_date = datetime.date(year,month,15)
            mid2_date = datetime.date(year,month,21)
            mid4_date = datetime.date(year,month,25)
            end_date = datetime.date(year,month,28)
            date_array.append([start_date.day, start_date.month, start_date.year])
            date_array.append([mid3_date.day, mid3_date.month, mid3_date.year])
            date_array.append([mid1_date.day, mid1_date.month, mid1_date.year])
            date_array.append([mid_date.day, mid_date.month, mid_date.year])
            date_array.append([mid2_date.day, mid2_date.month, mid2_date.year])
            date_array.append([mid4_date.day, mid4_date.month, mid4_date.year])
            date_array.append([end_date.day, end_date.month, end_date.year])

        return date_array




### Verification - PyKep algorithm

In [8]:
def VerPyKep(r_1, r_2, mu, t):
    # We solve the Lambert problem, using initial conditions given above
    l = pk.lambert_problem(r1=r_1, r2=r_2, tof=t, mu=mu, max_revs=0)

    # Extract velocity components
    v_1_ver = l.get_v1()[0]
    v_2_ver = l.get_v2()[0]

    v_1_ver_mag = magnitude(v_1_ver)
    v_2_ver_mag = magnitude(v_2_ver)

    return v_1_ver, v_2_ver, v_1_ver_mag, v_2_ver_mag


## Verification plots

In [9]:
# Departure date array
day_ref = [2023, 5, 9]  # REFERENCE DAY DETERMINED TO BE MAY 9 2023
day_ref_2 = [2025, 1, 1] # SECOND REFERENCE DAY, 1 JANUARY 2025 (FUTURE)
day_ref_3 = [2023, 7, 1]

tof_array = np.arange(1.0, 30.1, 0.1) * 30 * 24 * 3600  # s

def fun(day_ref):
    delta_V_ver = []
    delta_V = []
    diff = []
    r1 = []
    r2 = []
    for tof in tof_array:
        jd, fr = jday(
            day_ref[0], day_ref[1], day_ref[2], 0, 0, 0
        )  # Epoch determined at each 00:00:00 hr
        jd_arrival = jd + tof / 24 / 3600
        r_1, r_2 = PosVecs(jd, jd_arrival)
        r1.append(r_1)
        r2.append(r_2)
        v_1, v_2, v_1_mag, v_2_mag = LambertIzzo(r_1, r_2, mu_Sun, tof)
        delta_V.append((delta_V_Mars_to_Earth(v_1_mag, v_2_mag)) / 1000)  # km/s
        v_1_ver, v_2_ver, v_1_mag_ver, v_2_mag_ver = VerPyKep(r_1, r_2, mu_Sun, tof)
        delta_V_ver.append(
            (delta_V_Mars_to_Earth(v_1_mag_ver, v_2_mag_ver)) / 1000
        )  # km/s

    plt.figure()
    plt.rcParams.update({"font.size": 16})
    plt.plot(tof_array / 24 / 3600, delta_V)
    plt.xlabel("Time of flight in days since " + str(day_ref))
    plt.ylabel("Required $\Delta$V (km/s)")
    plt.yscale('log')
    plt.title("Algorithm, required $\Delta$V versus time of flight")
    plt.show()

    plt.figure()
    plt.rcParams.update({"font.size": 16})
    plt.plot(tof_array / 24 / 3600, delta_V_ver)
    plt.xlabel("Time of flight in days since " + str(day_ref))
    plt.ylabel("Required $\Delta$V (km/s)")
    plt.yscale('log')
    plt.title("PyKep, required $\Delta$V versus time of flight")
    plt.show()

    plt.figure()
    plt.rcParams.update({"font.size": 16})
    plt.plot(tof_array / 24 / 3600, np.abs(np.array(delta_V_ver) - np.array(delta_V)))
    plt.xlabel("Time of flight in days since " + str(day_ref))
    plt.ylabel("$\epsilon (\Delta$V) (km/s)")
    plt.yscale("log")
    plt.title(
        "Absolute difference, PyKep"
    )
    plt.show()

    print(np.max(np.abs(np.array(delta_V_ver) - np.array(delta_V))))

    return delta_V

def PlotTuDatPy():
    delta_V_23 = fun(day_ref)
    delta_V_25 = fun(day_ref_2)

    tudat_23 = []
    tudat_25 = []

    v_1_tudat_23 = np.loadtxt("Cartesian_State_Files/v_1_results_TuDatPy_2023.dat")
    v_2_tudat_23 = np.loadtxt("Cartesian_State_Files/v_2_results_TuDatPy_2023.dat")
    v_1_tudat_25 = np.loadtxt("Cartesian_State_Files/v_1_results_TuDatPy_2025.dat")
    v_2_tudat_25 = np.loadtxt("Cartesian_State_Files/v_2_results_TuDatPy_2025.dat")

    for i in range(len(delta_V_23)):
        tudat_23.append((delta_V_Mars_to_Earth(magnitude(v_1_tudat_23[i,:]), magnitude(v_2_tudat_23[i,:]))) / 1000)
        tudat_25.append((delta_V_Mars_to_Earth(magnitude(v_1_tudat_25[i,:]), magnitude(v_2_tudat_25[i,:]))) / 1000)

    plt.figure()
    plt.rcParams.update({"font.size": 16})
    plt.plot(tof_array / 24 / 3600, tudat_23)
    plt.xlabel("Time of flight in days since [2023, 5, 9]")
    plt.ylabel("Required $\Delta$V (km/s)")
    plt.yscale('log')
    plt.title("TuDatPy, required $\Delta$V versus time of flight")
    plt.show()

    plt.figure()
    plt.rcParams.update({"font.size": 16})
    plt.plot(tof_array / 24 / 3600, tudat_25)
    plt.xlabel("Time of flight in days since [2025, 1, 1]")
    plt.ylabel("Required $\Delta$V (km/s)")
    plt.yscale('log')
    plt.title("TuDatPy, required $\Delta$V versus time of flight")
    plt.show()

    plt.figure()
    plt.rcParams.update({"font.size": 16})
    plt.plot(tof_array / 24 / 3600, np.abs(np.array(tudat_23) - np.array(delta_V_23)))
    plt.xlabel("Time of flight in days since [2023, 5, 9]")
    plt.ylabel("$\epsilon (\Delta$V) (km/s)")
    plt.yscale('log')
    plt.title(
        "Absolute difference, TuDatPy"
    )
    plt.show()

    plt.figure()
    plt.rcParams.update({"font.size": 16})
    plt.plot(tof_array / 24 / 3600, np.abs(np.array(tudat_25) - np.array(delta_V_25)))
    plt.xlabel("Time of flight in days since [2025, 1, 1]")
    plt.ylabel("$\epsilon (\Delta$V) (km/s)")
    plt.yscale('log')
    plt.title(
        "Absolute difference, TuDatPy"
    )
    plt.show()

    return np.max(np.abs(np.array(tudat_23) - np.array(delta_V_23))), np.max(np.abs(np.array(tudat_25) - np.array(delta_V_25)))




## Porkchop

In [10]:

def PorkChop2(year):
    # Time of Flight array, in seconds
    tof_array = np.linspace(150, 630, num = 121) * 24 * 3600
    # Departure days, take every day in 2024
    departure_dates = np.vstack((datesyear(year, 5)))
    # Create empty lists for storage
    v_inf_Mars_array = []
    v_inf_Earth_array = []
    julian_departure_dates = []
    julian_arrival_dates = []
    delta_v_tot_array = []
    
    # Start iterations over the entire departure date array
    for date in departure_dates:
        print(date)
        jd, fr = jday(date[2], date[1], date[0], 0 , 0, 0)
        julian_departure_dates.append(jd)
        for tof in tof_array:
            jd_arrival = jd + tof / 24 / 3600
            julian_arrival_dates.append(jd_arrival)
            r_1, r_2 = PosVecs(jd, jd_arrival)
            v_Mars, v_Earth = VelVecs(jd, jd_arrival)
            v_1, v_2, v_1_mag, v_2_mag = LambertIzzo(r_1, r_2, mu_Sun, tof)
            delta_v_1, delta_v_2 = delta_V_Mars_to_Earth_corrected(jd, jd_arrival, v_1, v_2, 3)

            ''' 
            Commented part below can be switched to represent delta_v, v_1/v_2 or v_inf
            '''

            v_inf_Mars_array.append((magnitude(v_1 - v_Mars) / 1000)**2) # Alteration, did not include the Mars and Earth velocities to display v1 and v2
            v_inf_Earth_array.append((magnitude(v_2 - v_Earth) / 1000)**2) # Alteration: C3 values by taking square, convert to km first
            delta_v_tot_array.append((delta_v_1 + delta_v_2) / 1000) # Third plot with total Delta-V values of the trip, in km/s
    
    julian_departure_dates -= julian_departure_dates[0]

    v_inf_Mars_array = np.reshape(np.array(v_inf_Mars_array), (len(tof_array), len(departure_dates)), order = 'F')
    v_inf_Earth_array = np.reshape(np.array(v_inf_Earth_array), (len(tof_array), len(departure_dates)), order = 'F')
    delta_v_tot_array = np.reshape(np.array(delta_v_tot_array), (len(tof_array), len(departure_dates)), order = 'F')

    X, Y = np.meshgrid(julian_departure_dates, tof_array / 24 / 3600)
    Z1 = v_inf_Mars_array
    Z2 = v_inf_Earth_array
    Z3 = delta_v_tot_array

    levels_c3 = np.array([4, 8, 16, 18, 20, 22, 25, 30, 50, 75, 150, 300])

    plt.figure(1)
    plt.rcParams.update({"font.size": 16})
    plt.figure(figsize=(10, 10))
    contour_plot = plt.contour(X , Y, Z1, levels_c3)
    plt.clabel(contour_plot, levels_c3,
     inline=True, fontsize = 12, colors = 'black')

    # Add a colorbar to the plot
    colorbar = plt.colorbar(contour_plot, orientation="vertical")
    colorbar.ax.tick_params(width=10)

    # Set the labels for the axes and colorbar
    # plt.xlim(130, 320)
    plt.xlabel("Departure (days from 1-1-" +str(year)+")")
    plt.ylabel("Time of Flight (days)")
    plt.title("Porkchop plot Mars-to-Earth transfer, departure")
    plt.legend()
    plt.grid()
    # colorbar.ax.set_ylabel("$V_1$ [km/s]")
    # colorbar.ax.set_ylabel("$V_{\infty, dep}$ [km/s]")
    # colorbar.ax.set_ylabel("$\Delta V_1$ [km/s]")
    colorbar.ax.set_ylabel("$C_3$ values in $km^2/s^2$")
    plt.show()

    plt.figure(2)
    plt.figure(figsize=(10, 10))
    contour_plot = plt.contour(X, Y, Z2, levels_c3)
    plt.clabel(contour_plot, levels_c3,
     inline=True, fontsize = 12, colors = 'black')

    # Add a colorbar to the plot
    colorbar = plt.colorbar(contour_plot, orientation="vertical")
    colorbar.ax.tick_params(width=10)

    # Set the labels for the axes and colorbar
    # plt.ylim(150, 300)
    plt.xlabel("Departure (days from 1-1-" +str(year)+")")
    plt.ylabel("Time of Flight (days)")
    plt.title("Porkchop plot Mars-to-Earth transfer, arrival")
    plt.legend()
    plt.grid()
    # colorbar.ax.set_ylabel("$V_2$ [km/s]")
    # colorbar.ax.set_ylabel("$V_{\infty, arr}$ [km/s]")
    # colorbar.ax.set_ylabel("$\Delta V_2$ [km/s]")
    colorbar.ax.set_ylabel("$C_3$ values in $km^2/s^2$")
    plt.show()

    levels_delta_v = np.linspace(0, 100, 26)

    plt.figure(3)
    plt.figure(figsize=(10, 10))
    contour_plot = plt.contour(X, Y, Z3, levels_delta_v)
    plt.clabel(contour_plot, levels_delta_v,
     inline=True, fontsize = 12, colors = 'black')

    # Add a colorbar to the plot
    colorbar = plt.colorbar(contour_plot, orientation="vertical")
    colorbar.ax.tick_params(width=10)

    # Set the labels for the axes and colorbar
    # plt.ylim(150, 300)
    plt.xlabel("Departure (days from 1-1-" +str(year)+")")
    plt.ylabel("Time of Flight (days)")
    plt.title("Porkchop plot Mars-to-Earth transfer, total $\Delta V$")
    plt.legend()
    plt.grid()
    # colorbar.ax.set_ylabel("$V_2$ [km/s]")
    # colorbar.ax.set_ylabel("$V_{\infty, arr}$ [km/s]")
    # colorbar.ax.set_ylabel("$\Delta V_2$ [km/s]")
    colorbar.ax.set_ylabel("$\Delta V_{tot}$ values in $km/s$")
    plt.show()

    return "OK"





In [11]:
"""
INCORRECT
This cell contains the code for porckhop plot generation. Three functions are displayed.

PorkChop2(): Combine functions below that are incorrect, this will be updated. Steps to be taken are:
- Generate delta_V values for departure date and tof (correct delta_V value check)
- Order the delta_V values so that they exactly correspond to departure date (CHECK)
- Make the contour plot

PorkChopValues(): Computes all necessary values for the final \Delta V needed for the entire journey
between Mars and the Earth. These values are saved to local files.

PorkChopPlot(): This function generates the porkchop plot for the computed time scale.
It represents the total \Delta V of the mission on the z-axis.

"""



def PorkChopValues():
    tof_array = np.arange(5.0, 15.1, 0.1) * 30 * 24 * 3600  # s
    t_years = np.arange(2023, 2025, 1)
    t_months = np.arange(1, 13, 1)
    t_days = np.array([1, 15])

    departure_date_array = []
    julian_dates = []
    delta_V = []
    V_1 = []
    V_2 = []

    for i in t_years:
        for j in t_months:
            for k in t_days:
                departure_date_array.append([i, j, k])

    for day in departure_date_array:
        jd, fr = jday(
            day[0], day[1], day[2], 0, 0, 0
        )  # Epoch determined at each 00:00:00 hr
        julian_dates.append(jd)
        for tof in tof_array:
            jd_arrival = jd + tof / 24 / 3600
            r_1, r_2 = PosVecs(jd, jd_arrival)
            v_1, v_2, v_1_mag, v_2_mag = LambertIzzo(r_1, r_2, mu_Sun, tof)
            delta_V.append(delta_V_Mars_to_Earth(v_1_mag, v_2_mag))
            V_1.append(v_1_mag)
            V_2.append(v_2_mag)

    np.reshape(delta_V, [len(julian_dates), len(tof_array)])

    # Save results for easy porkchop plot generation
    np.savetxt("porkchop_output/single_lambert_delta_V.txt", delta_V)
    np.savetxt("porkchop_output/single_lambert_julian_dates.txt", julian_dates)
    np.savetxt("porkchop_output/single_lambert_tof.txt", tof_array)
    np.savetxt("porkchop_output/single_lambert_v1.txt", V_1)
    np.savetxt("porkchop_output/single_lambert_v2.txt", V_2)

    return "OK"


def PorkChopPlot():
    launch_dates = np.loadtxt("porkchop_output/single_lambert_julian_dates.txt") - np.loadtxt("porkchop_output/single_lambert_julian_dates.txt")[0]
    time_of_flight = np.loadtxt("porkchop_output/single_lambert_tof.txt") / 24 / 3600
    delta_V = np.loadtxt("porkchop_output/single_lambert_delta_v.txt")

    c = np.reshape(np.array(delta_V), (len(time_of_flight), len(launch_dates)), order = 'F')[::-1, :]

    X, Y = np.meshgrid(launch_dates, time_of_flight)

    plt.figure(1)
    # plt.figure(figsize=(10, 15))

    contour_plot = plt.contourf(X, Y, c / 1000, 8)
    plt.clabel(contour_plot, levels = [5,10,12.5, 17.5],
     inline=True, fontsize = 12, colors = 'black')


    # Add a colorbar to the plot
    colorbar = plt.colorbar(contour_plot, orientation="vertical")
    colorbar.ax.tick_params(width=10)

    # Set the labels for the axes and colorbar
    # plt.ylim(150, 300)
    plt.xlabel("Departure (days from 1-1-2023)")
    plt.ylabel("Arrival (days from 1-1-2023)")
    plt.title("Pork chop plot for given start dates and arrival dates from 01-01-2023")
    plt.legend()
    colorbar.ax.set_ylabel("Delta V (km/s)")
    plt.show()

    return "OK"



## Solar System Visual

In [12]:
"""
VisSolarSystem2023(): This function produces a plot with the positions of the Earth, Mars and Venus with respect
to the Sun at 1-1-2023. Moreover, their trajectories are plotted for the year 2023.
Can potentially be used for later visual representation of the problem

VisSolarSystemDate(jd, jd_arrival, jd_arrival_2): This function takes as input arguments the departure julian date 
from Mars, the arrival julian date at Venus and then the arrival julian date at Earth. With these dates, the 
planet positions at each of the dates are given along with their trajectories during the entire time of flight
"""

def VisSolarSystem2023():
    time_year = np.arange(2023, 2024, 1)
    time_months = np.arange(1, 13, 1)
    time_days = np.arange(1, 31, 15)
    time_array = []

    for i in time_year:
        for j in time_months:
            for k in time_days:
                time_array.append([i,j,k])

    Earth = []
    Mars = []
    Venus = []
    for date in time_array:

        jd, fr = jday(date[0], date[1], date[2], 0, 0, 0)

        # Define Mars epoch, CURRENT DAY 00:00:00
        obj_Mars = Horizons(id="499", location="500@10", epochs=jd)

        eph_Mars = obj_Mars.vectors()

        Mars.append(eph_Mars["x", "y", "z"])

        # Define Earth epoch, CURRENT DAY 00:00:00
        obj_Earth = Horizons(id="399", location="500@10", epochs=jd)

        eph_Earth = obj_Earth.vectors()

        Earth.append(eph_Earth["x", "y", "z"])

        # Define Venus epoch, CURRENT DAY 00:00:00
        obj_Venus = Horizons(id="299", location="500@10", epochs=jd)

        eph_Venus = obj_Venus.vectors()

        Venus.append(eph_Venus["x", "y", "z"])

        # Convert to arrays

    Mars = np.array(Mars)
    Venus = np.array(Venus)
    Earth = np.array(Earth)

    # Create figure

    # 3D plot over 2023
    # Positions indicated on 1-1-2023

    plt.figure()
    ax = plt.axes(projection = '3d')
    ax.plot3D(Earth['x'], Earth['y'], Earth['z'], label = 'Earth', linestyle = ':', color = 'royalblue')
    ax.plot3D(Mars['x'], Mars['y'], Mars['z'], label = 'Mars', linestyle = ':', color = 'darkred')
    ax.plot3D(Venus['x'], Venus['y'], Venus['z'], label = 'Venus', linestyle = ':', color = 'olive')
    ax.scatter(Earth[0]['x'], Earth[0]['y'], Earth[0]['z'], color='royalblue', s=100, label = 'Earth')
    ax.scatter(Mars[0]['x'], Mars[0]['y'], Mars[0]['z'], color='darkred', s=50, label = 'Mars')
    ax.scatter(Venus[0]['x'], Venus[0]['y'], Venus[0]['z'], color='olive', s=25, label = 'Venus')
    # Plot the Sun as a large yellow orb at [0, 0, 0]
    ax.scatter([0], [0], [0], color='yellow', s=200, label='Sun')
    ax.set_title("3D plot of Mars, Earth and Venus orbits over 2023")
    ax.set_xlabel("x [AU]")
    ax.set_ylabel("y [AU]")
    ax.set_zlabel("z [AU]")
    ax.set_zlim(-1,1)
    ax.legend()
    plt.show()

    # 2D plot over 2023

    plt.figure()
    ax = plt.axes(projection = '3d')
    ax.plot3D(Earth['x'], Earth['y'], 0, label = 'Earth', linestyle = ':', color = 'royalblue')
    ax.plot3D(Mars['x'], Mars['y'], 0, label = 'Mars', linestyle = ':', color = 'darkred')
    ax.plot3D(Venus['x'], Venus['y'], 0, label = 'Venus', linestyle = ':', color = 'olive')
    ax.scatter(Earth[0]['x'], Earth[0]['y'], 0, color='royalblue', s=100, label = 'Earth')
    ax.scatter(Mars[0]['x'], Mars[0]['y'], 0 , color='darkred', s=50, label = 'Mars')
    ax.scatter(Venus[0]['x'], Venus[0]['y'], 0, color='olive', s=25, label = 'Venus')
    # Plot the Sun as a large yellow orb at [0, 0, 0]
    ax.scatter([0], [0], [0], color='yellow', s=200, label='Sun')
    ax.set_title("2D plot of Mars, Earth and Venus orbits over 2023")
    ax.set_xlabel("x [AU]")
    ax.set_ylabel("y [AU]")
    ax.set_zlabel("z [AU]")
    ax.legend()
    plt.show()

    return Mars[1]['x']

def VisSolarSystemDate(jd, jd_arrival, jd_arrival_2):
    Earth = []
    Mars = []
    Venus = []

    # Define Mars epoch, jd, CURRENT DAY 00:00:00
    obj_Mars = Horizons(id="499", location="500@10", epochs=jd)

    eph_Mars = obj_Mars.vectors()

    pos_Mars = eph_Mars["x","y","z"]

    # Venus: iterate from jd towardes jd_arrival
    for day in np.arange(jd, jd_arrival, 1):
        obj_Venus = Horizons(id="299", location="500@10", epochs=day)

        eph_Venus = obj_Venus.vectors()

        Venus.append(eph_Venus["x", "y", "z"])
    
    # Earth: iterate from jd towards jd_arrival_2
    for day2 in np.arange(jd, jd_arrival_2, 1):
        obj_Earth = Horizons(id="399", location="500@10", epochs=day2)

        eph_Earth = obj_Earth.vectors()

        Earth.append(eph_Earth["x", "y", "z"])
    
    Venus = np.array(Venus)
    Earth = np.array(Earth)

    # Create plot    
    plt.figure()
    ax = plt.axes(projection = '3d')
    ax.plot3D(Earth['x'], Earth['y'], Earth['z'], linestyle = ':', color = 'royalblue')
    ax.plot3D(Venus['x'], Venus['y'], Venus['z'], linestyle = ':', color = 'olive')
    ax.scatter(Earth[-1]['x'], Earth[-1]['y'], Earth[-1]['z'], color='royalblue', s=100, label = 'Earth arrival position')
    ax.scatter(pos_Mars['x'], pos_Mars['y'], pos_Mars['z'], color='darkred', s=50, label = 'Mars departure position')
    ax.scatter(Venus[-1]['x'], Venus[-1]['y'], Venus[-1]['z'], color='olive', s=25, label = 'Venus arrival position')
    # Plot the Sun as a large yellow orb at [0, 0, 0]
    ax.scatter([0], [0], [0], color='yellow', s=200, label='Sun')
    ax.set_title("3D plot of Mars, Earth and Venus orbits during transfer.")
    ax.set_xlabel("x [AU]")
    ax.set_ylabel("y [AU]")
    ax.set_zlabel("z [AU]")
    ax.set_zlim(-1,1)
    # ax.legend(loc = 'right')
    plt.legend()
    plt.show()
    
    return "OK"




## Gravity Assist Analysis

In [13]:
"""
NEW ALGORITHM STARTED
Use vectors instead here, delta_V_Mars_to_Venus has been corrected


VenusGA(r_3, 2d3d): this function will be solely dependent on the perigee distance of the Venus flyby
The first leg of the trajectory will be performed with a predefined Lambert problem (potential ToF parameter)
The Gravity Assist will then ben analysed
The last leg will be fitted using the Lambert targeter, optimization problem. This will then adhere to specific
ToF as well.
2D AND 3D OPTION INCORPORATED, dim == 2 gives dim == 3 gives 3d

Outputs: total delta_v of entire trajectory, total ToF of entire trajectory
"""


def VenusGA(r_flyby, ref_date, tof_1, tof_2, dim):
    # Distinguish entire analysis between 2D and 3D
    if dim == 2:
        # First leg
        jd, fr = jday(ref_date[0], ref_date[1], ref_date[2], 0, 0, 0)
        jd_arrival = jd + tof_1 / 24 / 3600
        r_1, r_2 = PosVecsVenus(jd, jd_arrival)
        r_1[2] = 0
        r_2[2] = 0
        v_1, v_2, v_1_mag, v_2_mag = LambertIzzo(r_1, r_2, mu_Sun, tof_1)
        delta_v_1, v_inf_2 = delta_V_Mars_to_Venus(jd, jd_arrival, v_1, v_2, 2)

        # Second leg, time of flight and Earth positions
        jd_arrival_2 = jd_arrival + tof_2 / 24 / 3600
        r_3, r_4 = PosVecsEarth(jd_arrival, jd_arrival_2)
        r_3[2] = 0
        r_4[2] = 0
        r_ref_plane = r_4 - r_3
        l_unit_vector = np.cross(v_inf_2, r_ref_plane) / magnitude(
            np.cross(v_inf_2, r_ref_plane)
        )

        # Gravity Assist
        v_Venus = VelVecsVenus(jd, jd_arrival)[1]
        v_Venus[2] = 0
        e = 1 + r_flyby * magnitude(v_inf_2) ** 2 / mu_Venus
        delta = 2 * np.arccos(-1 / e) - np.pi

        # Delta v calculation in vector form, from thesis work, USE DELTA FORMULATION = CORRECT

        delta_v_GA = (
            2
            / e
            * (
                -v_inf_2 * np.sin(delta / 2)
                + np.cross(l_unit_vector, v_inf_2) * np.cos(delta / 2)
            )
        )

        # Return v_3 (heliocentric velocity at Venus) for Lambert comparison

        v_3 = v_2 + delta_v_GA  # m/s

        return v_3

    elif dim == 3:
        # First leg
        jd, fr = jday(ref_date[0], ref_date[1], ref_date[2], 0, 0, 0)
        jd_arrival = jd + tof_1 / 24 / 3600
        r_1, r_2 = PosVecsVenus(jd, jd_arrival)
        v_1, v_2, v_1_mag, v_2_mag = LambertIzzo(r_1, r_2, mu_Sun, tof_1)
        delta_v_1, v_inf_2 = delta_V_Mars_to_Venus(jd, jd_arrival, v_1, v_2, 3)

        # Second leg, time of flight and Earth positions
        jd_arrival_2 = jd_arrival + tof_2 / 24 / 3600
        r_3, r_4 = PosVecsEarth(jd_arrival, jd_arrival_2)

        # Gravity Assist
        v_Mars, v_Venus = VelVecsVenus(jd, jd_arrival)

        v_perigee = np.sqrt(
            magnitude(v_inf_2) ** 2 + 2 * mu_Venus / r_flyby
        )  # MAGNITUDE
        sma = -mu_Venus / magnitude(v_inf_2) ** 2
        semi_latus = (r_flyby * v_perigee) ** 2 / mu_Venus
        eps = np.sqrt(1 - semi_latus / sma)
        delta = 2 * np.arcsin(1 / eps)

        # Maximum and minimum delta values

        # 3D extra components
        # B-Plane orientation
        b_vec = np.cross(v_2, v_Venus) / magnitude(np.cross(v_2, v_Venus))
        u_inf = v_inf_2 / (magnitude(v_inf_2))
        c_vec = np.cross(b_vec, u_inf)

        # phi rotation angle
        # Defined by the planes and assumed ToF
        normal_1 = np.cross(r_1, r_2) / magnitude(np.cross(r_1, r_2))
        normal_2 = np.cross(r_3, r_4) / magnitude(np.cross(r_3, r_4))
        phi_ana = np.arccos(
            np.dot(normal_1, normal_2) / (magnitude(normal_1) * magnitude(normal_2))
        )

        dot = []
        phi_range = np.deg2rad(np.linspace(-180, 180, 360001))
        v_3_array = []
        for phi in phi_range:
            # Outgoing planetocentric velocity vector
            v_inf_3 = magnitude(v_inf_2) * (
                u_inf * np.cos(delta)
                + c_vec * np.sin(delta) * np.cos(phi)
                + b_vec * np.sin(delta) * np.sin(phi)
            )
            v_3 = v_inf_3 + v_Venus
            v_3_array.append(v_3)
            v_u_3 = v_3 / magnitude(v_3)
            dot.append(np.abs(np.dot(v_u_3, normal_2)))

        dot = np.array(dot)
        phi_range = np.array(phi_range)

        delta_v_GA = v_3_array[np.argmin(dot)] - v_2

        return (
            v_3_array[np.argmin(dot)],
            r_3,
            jd_arrival,
            jd_arrival_2,
            magnitude(delta_v_GA),
            delta_v_1,
            v_1 - v_Mars,
            v_inf_2,
            np.rad2deg(delta),
            np.rad2deg(phi_range[np.argmin(dot)]),
        )

    else:
        return print("INCORRECT DIMENSIONS")


# INITIATE LISTS FOR DATA STORAGE
r_range_array = np.arange(492e3, 502e3, 1e3)
# r_range_array = np.arange(492e3, 493e3, 1e3)
r_4_list = []
v_list = []
v_inf_list = []
geometry_list = []
diff_list = []

ref_date_final = [2029, 5, 6]
jd0, fr = jday(2029, 5, 6, 0, 0, 0)
tof_1 = 180 * 24 * 3600
tof_2 = 77.8 * 24 * 3600

for r in r_range_array:
    print(r)
    v_3, r_3, jd, jd2, delta_v_GA, delta_v_1, v_inf_1, v_inf_2, delta, phi = VenusGA(
        R_Venus + r, ref_date_final, tof_1, tof_2, 3
    )
    v_r_3 = np.dot(v_3, r_3) / magnitude(r_3)
    h_3 = np.cross(r_3, v_3)
    incl = np.arccos(h_3[2] / magnitude(h_3))
    N_array = np.cross(np.array([0, 0, 1]), h_3)
    Omega = np.arccos(N_array[0] / magnitude(N_array))
    if N_array[1] < 0:
        Omega = 2 * np.pi - Omega
    ecc_vec = (
        1
        / mu_Sun
        * (
            (magnitude(v_3) ** 2 - mu_Sun / magnitude(r_3)) * r_3
            - (np.dot(r_3, v_3) * v_3)
        )
    )
    omega = np.arccos(
        np.dot(N_array, ecc_vec) / (magnitude(N_array) * magnitude(ecc_vec))
    )
    if ecc_vec[2] < 0:
        omega = 2 * np.pi - omega
    theta = np.arccos(np.dot(ecc_vec, r_3) / (magnitude(ecc_vec) * magnitude(r_3)))
    if v_r_3 < 0:
        theta = 2 * np.pi - theta
    r_per = magnitude(h_3) ** 2 / mu_Sun / (1 + magnitude(ecc_vec) * np.cos(0))
    r_ap = magnitude(h_3) ** 2 / mu_Sun / (1 + magnitude(ecc_vec) * np.cos(np.pi))
    sma = (
        1 / 2 * (r_per + r_ap)
    )  # SMA CHECK FOR THIS SETUP > AU, LARGER SEMI-MAJOR AXIS FOR SMALLER FLYBY DISTANCE AND ALSO FOR NO FLYBY DISTANCE..?

    # Now, analyze the point where r = AU
    r_Venus, r_Earth = PosVecsEarth(jd, jd2)
    v_Venus, v_Earth = VelVecsEarth(jd, jd2)
    v_inf_3 = v_3 - v_Venus
    r_4_mag = magnitude(r_Earth)
    theta_2 = np.arccos(
        sma * (1 - magnitude(ecc_vec) ** 2) / (r_4_mag * magnitude(ecc_vec))
        - 1 / magnitude(ecc_vec)
    )
    # if np.dot(np.cross(N_array / magnitude(N_array), ecc_vec), h_3) < 0:
    #     theta_2 = -theta_2
    ksi = r_4_mag * np.cos(theta_2)
    eta = r_4_mag * np.sin(theta_2)

    # Necessary parts
    l_1 = np.cos(Omega) * np.cos(omega) - np.sin(Omega) * np.sin(omega) * np.cos(incl)
    l_2 = -np.cos(Omega) * np.sin(omega) - np.sin(Omega) * np.cos(omega) * np.cos(incl)
    m_1 = np.sin(Omega) * np.cos(omega) + np.cos(Omega) * np.sin(omega) * np.cos(incl)
    m_2 = -np.sin(Omega) * np.sin(omega) + np.cos(Omega) * np.cos(omega) * np.cos(incl)
    n_1 = np.sin(omega) * np.sin(incl)
    n_2 = np.cos(omega) * np.sin(incl)

    # Construct final positions
    r_4 = np.array([[l_1, l_2], [m_1, m_2], [n_1, n_2]]) @ np.array([ksi, eta]).T
    r_4_list.append(r_4)

    # CHECK ON PROXIMITY
    abs_diff = magnitude(r_4 - r_Earth)
    diff_list.append((abs_diff, r_SI, r))

    v_4 = (
        mu_Sun
        / magnitude(h_3)
        * np.array([[-l_1, l_2], [-m_1, m_2], [-n_1, n_2]])
        @ np.array([np.sin(theta_2), magnitude(ecc_vec) + np.cos(theta_2)]).T
    )
    v_inf_4 = v_4 - v_Earth
    v_p_E = np.sqrt(magnitude(v_inf_4) ** 2 + 2 * mu_Earth / r_park_Earth)
    v_park_E = np.sqrt(mu_Earth / r_park_Earth)
    delta_v_4 = np.abs(v_p_E - v_park_E)
    delta_v_tot = delta_v_1 + delta_v_4

    v_list.append([delta_v_1, delta_v_GA, delta_v_4, delta_v_tot])
    v_inf_list.append(
        [magnitude(v_inf_1), magnitude(v_inf_2), magnitude(v_inf_3), magnitude(v_inf_4)]
    )
    geometry_list.append([delta, phi, r])

    print("Delta_V needed for Mars departure is " + str(delta_v_1 / 1000) + " [km/s]")
    print("Delta_V needed for Earth arrival is " + str(delta_v_4 / 1000) + " [km/s]")
    print("Total delta_V is " + str(delta_v_tot / 1000) + " [km/s]")
    print("Gravity-assisted delta_V is " + str(delta_v_GA / 1000) + " [km/s]")

"""COMPARISON"""
r_1_comp, r_2_comp = PosVecs(jd0, jd2)
v_1_comp, v_2_comp, v_1_mag, v_2_mag = LambertIzzo(
    r_1_comp, r_2_comp, mu_Sun, tof_1 + tof_2
)
delta_v_1_comp, delta_v_2_comp = delta_V_Mars_to_Earth_corrected(
    jd0, jd2, v_1_comp, v_2_comp, 2
)
total_delta_v_Lambert = delta_v_1_comp + delta_v_2_comp
print(
    "Delta_ 1 and 2 are "
    + str(delta_v_1_comp / 1000)
    + " and "
    + str(delta_v_2_comp / 1000)
    + " [km/s]"
)
print(
    "Total delta_V direct Lambert is " + str(total_delta_v_Lambert / 1000) + " [km/s]"
)

np.savetxt("Data/delta_v_list", v_list)
np.savetxt("Data/v_inf_list", v_inf_list)
np.savetxt("Data/geometry_list", geometry_list)
np.savetxt("Data/diff_list", diff_list)

x = np.array([point[0] for point in r_4_list]) / AU
y = np.array([point[1] for point in r_4_list]) / AU
z = np.array([point[2] for point in r_4_list]) / AU


def plot3DGA(x, y, z):
    plt.figure()
    ax = plt.axes(projection="3d")
    ax.plot3D(x, y, z, label="Potential hits", color="red")
    ax.scatter(
        r_Earth[0] / AU,
        r_Earth[1] / AU,
        r_Earth[2] / AU,
        s=int(R_Earth / AU),
        color="royalblue",
        label="Earth",
    )
    # ax.scatter([0], [0], [0], color='yellow', label='Sun')

    u = np.linspace(0, 2 * np.pi, 100)
    v = np.linspace(0, np.pi, 100)
    x_s = (r_SI * np.outer(np.cos(u), np.sin(v)) + r_Earth[0]) / AU
    y_s = (r_SI * np.outer(np.sin(u), np.sin(v)) + r_Earth[1]) / AU
    z_s = (r_SI * np.outer(np.ones(np.size(u)), np.cos(v)) + r_Earth[2]) / AU

    x_E = (R_Earth * np.outer(np.cos(u), np.sin(v)) + r_Earth[0]) / AU
    y_E = (R_Earth * np.outer(np.sin(u), np.sin(v)) + r_Earth[1]) / AU
    z_E = (R_Earth * np.outer(np.ones(np.size(u)), np.cos(v)) + r_Earth[2]) / AU

    ax.plot_wireframe(
        x_s,
        y_s,
        z_s,
        rstride=10,
        cstride=10,
        color="blue",
        linewidth=0.5,
        linestyle="dotted",
        label="SoI",
    )
    ax.plot_surface(x_E, y_E, z_E, color="green", label="Earth")

    ax.set_title("Potential hits within the flyby range given")
    ax.set_xlabel("x [AU]")
    ax.set_ylabel("y [AU]")
    ax.set_zlabel("z [AU]")
    # ax.set_xlim(-1.1, 1.1)
    # ax.set_ylim(-1.1, 1.1)
    # ax.set_zlim(-1.1, 1.1)
    ax.legend()
    plt.show()

    return x, y, z


"""
A forward Euler iteration will be used without perturbations to check what the final velocity
and position will be for given times of flight. With this, the chance of hitting the Earth is
approximated.
"""


def ForwardEulerSC(r_0, v_0, tof, jd_Earth):
    dt = 1
    time_array = np.arange(0, tof, dt)
    x_arr = []
    y_arr = []
    z_arr = []

    for time in time_array:
        x = r_0[0]
        y = r_0[1]
        z = r_0[2]
        vx = v_0[0]
        vy = v_0[1]
        vz = v_0[2]

        x_arr.append(x)
        y_arr.append(y)
        z_arr.append(z)

        # Second derivative
        x_dot_dot = -mu_Sun / (np.sqrt(x**2 + y**2 + z**2) ** 3) * x
        y_dot_dot = -mu_Sun / (np.sqrt(x**2 + y**2 + z**2) ** 3) * y
        z_dot_dot = -mu_Sun / (np.sqrt(x**2 + y**2 + z**2) ** 3) * z

        # First derivative
        x_dot = vx
        y_dot = vy
        z_dot = vz

        # Update state
        vx += x_dot_dot * dt
        vy += y_dot_dot * dt
        vz += z_dot_dot * dt
        x += x_dot * dt
        y += y_dot * dt
        z += z_dot * dt

        r_0 = np.array([x, y, z]).T
        v_0 = np.array([vx, vy, vz]).T

    # Make array ready for plot
    x_arr = np.array(x_arr) / AU
    y_arr = np.array(y_arr) / AU
    z_arr = np.array(z_arr) / AU

    nan, r_4 = PosVecsEarth(jd_Earth, jd_Earth)

    # # Plot
    # plt.figure()
    # ax = plt.axes(projection = '3d')
    # ax.plot3D(x_arr, y_arr, z_arr)
    # # Plot the Sun as a large yellow orb at [0, 0, 0]
    # ax.scatter(r_4[0] / AU, r_4[1] / AU, r_4[2] / AU, color='royalblue', s=100, label = 'Earth')
    # ax.scatter([0], [0], [0], color='yellow', s=200, label='Sun')
    # ax.set_title("3D plot of integrated transfer from Venus to Earth.")
    # ax.set_xlabel("x [AU]")
    # ax.set_ylabel("y [AU]")
    # ax.set_zlabel("z [AU]")
    # ax.set_zlim(-1,1)
    # # ax.legend(loc = 'right')
    # plt.legend()
    # plt.show()

    return magnitude(r_0 - r_4)


def eccentricity(r, v):
    return (
        1
        / mu_Sun
        * ((magnitude(v) ** 2 - mu_Sun / magnitude(r)) * r - np.dot(r, v) * v)
    )


def argper(r, v, e):
    omega = np.arctan2(e[1], e[0])

    if (np.cross(r, v)[2]) < 0:
        omega = 2 * np.pi - omega

    return omega


def trueanom(p, e_mag, r_mag):
    return np.arccos(p / (r_mag * e_mag) - 1 / e_mag)


def testfunction():
    # TEST INPUT VALUES
    refdate = [2024, 10, 11]
    jd, fr = jday(refdate[0], refdate[1], refdate[2], 0, 0, 0)
    tof1 = 365 * 3600 * 24
    tof2 = 20 * 3600 * 24

    v_3 = VenusGA(5 * r_flyby_Venus, refdate, tof1, tof2, 2)
    r_3, r_E = PosVecsEarth(jd + tof1 / 24 / 3600, jd + tof1 / 24 / 3600)
    v_V, v_E = VelVecsEarth(
        jd + tof1 / 24 / 3600, jd + tof1 / 24 / 3600
    )  # Evaluate at the same time

    """ Make 2D problem """
    r_3[2] = 0
    r_E[2] = 0
    v_V[2] = 0
    v_E[2] = 0

    r_3_hat = r_3 / magnitude(r_3)

    """ Spacecraft Keplerian elements """
    h_3 = np.cross(r_3, v_3)
    h_3_mag = magnitude(h_3)
    e_3 = eccentricity(r_3, v_3)
    e_3_mag = magnitude(e_3)
    e_3_hat = e_3 / e_3_mag
    i = np.arccos(h_3[2] / h_3_mag)  # CHECK = 0
    omega_3 = argper(r_3, v_3, e_3)
    p_3 = h_3_mag ** (2) / mu_Sun
    r_p_3 = p_3 / (1 + e_3_mag * np.cos(0))
    r_a_3 = p_3 / (1 + e_3_mag * np.cos(np.pi))
    a_3 = 1 / 2 * (r_p_3 + r_a_3)
    T_3 = 2 * np.pi / np.sqrt(mu_Sun) * a_3 ** (3 / 2)

    """ Earth Keplerian elements """
    h_E = np.cross(r_E, v_E)
    h_E_mag = magnitude(h_E)
    e_E = eccentricity(r_E, v_E)
    e_E_mag = magnitude(e_E)
    e_E_hat = e_E / e_E_mag
    p_E = h_E_mag**2 / mu_Sun
    r_p_E = p_E / (1 + e_E_mag * np.cos(0))
    r_a_E = p_E / mu_Sun / (1 + e_E_mag * np.cos(np.pi))
    a_E = 1 / 2 * (r_p_E + r_a_E)
    omega_E = argper(r_E, v_E, e_E)

    """
    From this point, assumption that the distance to the Earth from the Sun is exactly 1 AU, with this, theta matching
    """
    r_match = AU
    theta_E = trueanom(p_E, e_E_mag, r_match)
    theta_3 = trueanom(p_3, e_3_mag, r_match)

    print(theta_3)

    r_match_test = p_3 / (1 + e_3_mag * np.cos(theta_3))

    print(r_match_test / AU)
    nu_E = omega_E + theta_E
    nu_3 = omega_3 + theta_3

    print(np.rad2deg(nu_E), np.rad2deg(nu_3))

    return "OK"


def plottest(date, tof1, c):
    if c == 1:
        jd, fr = jday(date[0], date[1], date[2], 0, 0, 0)
        # tof1 = 320 * 3600 * 24
        tof2 = 20 * 3600 * 24

        r_array = np.linspace(R_Venus + 200 * 10**3, 5 * R_Venus, 100)
        nu_3 = []
        nu_E = []
        for flyby in r_array:
            v_3 = VenusGA(flyby, date, tof1, tof2, 2)
            r_3, r_E = PosVecsEarth(jd + tof1 / 24 / 3600, jd + tof1 / 24 / 3600)
            v_V, v_E = VelVecsEarth(
                jd + tof1 / 24 / 3600, jd + tof1 / 24 / 3600
            )  # Evaluate at the same time
            e_3 = eccentricity(r_3, v_3)
            e_E = eccentricity(r_E, v_E)
            p_3 = magnitude(np.cross(r_3, v_3)) ** 2 / mu_Sun
            p_E = magnitude(np.cross(r_E, v_E)) ** 2 / mu_Sun
            omega_3 = argper(r_3, v_3, e_3)
            omega_E = argper(r_E, v_E, e_E)
            theta_3 = trueanom(p_3, magnitude(e_3), AU)
            theta_E = trueanom(p_E, magnitude(e_E), AU)
            nu_3.append(np.rad2deg(omega_3 + theta_3))
            nu_E.append(np.rad2deg(omega_E + theta_E))

        diff = np.abs(np.array(nu_3) - np.array(nu_E))
        r_flyby = r_array[np.argmin(diff)]
        v_3 = VenusGA(r_flyby, date, tof1, tof2, 2)
        r_3, r_E = PosVecsEarth(jd + tof1 / 24 / 3600, jd + tof1 / 24 / 3600)
        v_V, v_E = VelVecsEarth(
            jd + tof1 / 24 / 3600, jd + tof1 / 24 / 3600
        )  # Evaluate at the same time
        e_3 = eccentricity(r_3, v_3)
        e_E = eccentricity(r_E, v_E)
        p_3 = magnitude(np.cross(r_3, v_3)) ** 2 / mu_Sun
        a_3 = p_3 / (1 - magnitude(e_3) ** 2)
        p_E = magnitude(np.cross(r_E, v_E)) ** 2 / mu_Sun
        omega_3 = argper(r_3, v_3, e_3)
        omega_E = argper(r_E, v_E, e_E)
        theta_2 = trueanom(p_3, magnitude(e_3), magnitude(r_3))
        theta_3 = trueanom(
            p_3, magnitude(e_3), AU
        )  # Note that this is theta at arrival
        theta_E = trueanom(p_E, magnitude(e_E), AU)

        # Velocity component calculation
        v_r_4 = (
            mu_Sun / magnitude(np.cross(r_3, v_3)) * magnitude(e_3) * np.sin(theta_3)
        )
        v_th_4 = (
            mu_Sun
            / magnitude(np.cross(r_3, v_3))
            * (1 + magnitude(e_3) * np.cos(theta_3))
        )
        v_x_4 = v_r_4 * np.cos(theta_3) - v_th_4 * np.sin(theta_3)
        v_y_4 = v_r_4 * np.sin(theta_3) + v_th_4 * np.cos(theta_3)
        v_4 = np.array([v_x_4, v_y_4, 0])

        # Calculate Tof_2
        E_2 = 2 * np.arctan(
            np.sqrt((1 - magnitude(e_3)) / (1 + magnitude(e_3))) * np.tan(theta_2 / 2)
        )
        E_3 = 2 * np.arctan(
            np.sqrt((1 - magnitude(e_3)) / (1 + magnitude(e_3))) * np.tan(theta_3 / 2)
        )
        M_2 = E_2 - magnitude(e_3) * np.sin(E_2)
        M_3 = E_3 - magnitude(e_3) * np.sin(E_3)
        mean_motion = np.sqrt(mu_Sun / a_3**3)
        tof_2 = M_3 / mean_motion - M_2 / mean_motion

        plt.figure(1)
        plt.plot(r_array, nu_3, label="Spacecraft")
        plt.plot(r_array, nu_E, label="Earth")
        plt.vlines(
            R_Venus,
            ymin=0,
            ymax=360,
            colors="red",
            linestyles="dashed",
            label="Venus radius",
        )
        plt.ylabel("True latitude [deg]")
        plt.xlabel("Flyby distance [m]")
        plt.title(str(date) + " tof_1 = " + str(tof1 / 24 / 3600) + " days")
        plt.legend()
        plt.show()

    elif c == 2:
        jd, fr = jday(date[0], date[1], date[2], 0, 0, 0)
        # tof1 = np.linspace(30, 400, 38)
        tof2 = 20 * 3600 * 24

        r_flyby = R_Venus
        nu_3 = []
        nu_E = []
        for tof in tof1:
            v_3 = VenusGA(flyby, date, tof, tof2, 2)
            r_3, r_E = PosVecsEarth(jd + tof / 24 / 3600, jd + tof / 24 / 3600)
            v_V, v_E = VelVecsEarth(
                jd + tof / 24 / 3600, jd + tof / 24 / 3600
            )  # Evaluate at the same time
            e_3 = eccentricity(r_3, v_3)
            e_E = eccentricity(r_E, v_E)
            p_3 = np.cross(r_3, v_3) ** 2 / mu_Sun
            p_E = np.cross(r_E, v_E) ** 2 / mu_Sun
            omega_3 = argper(r_3, v_3, e_3)
            omega_E = argper(r_E, v_E, e_3)
            theta_3 = trueanom(p_3, magnitude(e_3), AU)
            theta_E = trueanom(p_E, magnitude(e_E), AU)
            nu_3.append(omega_3 + theta_3)
            nu_E.append(omega_E + theta_E)

            # Velocity component calculation
            v_r_4 = (
                mu_Sun
                / magnitude(np.cross(r_3, v_3))
                * magnitude(e_3)
                * np.sin(theta_3)
            )
            v_th_4 = (
                mu_Sun
                / magnitude(np.cross(r_3, v_3))
                * (1 + magnitude(e_3) * np.cos(theta_3))
            )
            v_x_4 = v_r_4 * np.cos(theta_3) - v_th_4 * np.sin(theta_3)
            v_y_4 = v_r_4 * np.sin(theta_3) + v_th_4 * np.cos(theta_3)
            v_4 = np.array([v_x_4, v_y_4, 0])

        plt.figure(2)
        plt.plot(r_array, nu_3, label="nu_3")
        plt.plot(r_array, nu_E, label="nu_E")
        plt.xlabel("Flyby distance [m]")
        plt.ylabel("Omega plus theta value [rad]")
        plt.legend()
        plt.show()

    return v_3, v_4, tof_2, r_flyby


def gravityassistcheck():
    """
    Check if the gravity assist is beneficial
    """
    tof_1 = 180 * 24 * 3600  # seconds
    date_ref = [2029, 5, 4]
    jd, fr = jday(date_ref[0], date_ref[1], date_ref[2], 0, 0, 0)
    jd_arrival_1 = jd + tof_1 / 24 / 3600
    v_3, v_4, tof_2, r_flyby = plottest(date_ref, tof_1, 1)
    r_1, r_2 = PosVecsVenus(jd, jd_arrival_1)
    v_1, v_2, v_1_mag, v_2_mag = LambertIzzo(r_1, r_2, mu_Sun, tof_1)
    delta_v_1, v_inf_Venus = delta_V_Mars_to_Venus(jd, jd_arrival_1, v_1, v_2, 2)
    jd_arrival_2 = jd_arrival_1 + tof_2 / 3600 / 24
    v_Venus, v_Earth = VelVecsEarth(jd_arrival_1, jd_arrival_2)
    v_Venus[2] = 0
    v_Earth[2] = 0

    v_inf_4 = v_4 - v_Earth
    v_inf_4_mag = magnitude(v_inf_4)

    v_park_Earth = np.sqrt(mu_Earth / r_park_Earth)
    v_p_Earth = np.sqrt(v_inf_4_mag**2 + (2 * mu_Earth) / r_park_Earth)
    delta_v_4 = np.abs(v_p_Earth - v_park_Earth)

    total_delta_v_GA = delta_v_1 + delta_v_4

    """COMPARISON"""
    r_1_comp, r_2_comp = PosVecs(jd, jd_arrival_2)
    v_1_comp, v_2_comp, v_1_mag, v_2_mag = LambertIzzo(
        r_1_comp, r_2_comp, mu_Sun, tof_1 + tof_2
    )
    delta_v_1_comp, delta_v_2_comp = delta_V_Mars_to_Earth_corrected(
        jd, jd_arrival_2, v_1_comp, v_2_comp, 2
    )
    total_delta_v_Lambert = delta_v_1_comp + delta_v_2_comp

    VisSolarSystemDate(jd, jd_arrival_1, jd_arrival_2)
    print(
        total_delta_v_GA / 1000,
        total_delta_v_Lambert / 1000,
        (total_delta_v_Lambert - total_delta_v_GA) / 1000,
        jd_arrival_2 - jd,
        r_flyby - R_Venus,
    )

    return "OK"

492000.0


KeyboardInterrupt: 

In [16]:
# TEST

def report():
    date_array_report = [[2023, 1, 1], [2023, 5, 10], [2025, 11, 21], [2029, 10, 11]]
    time_of_flight_report = np.array([100, 250, 365, 400]) * 24 * 3600

    for i in range(0,len(date_array_report)):
        jd, fr = jday(date_array_report[i][0], date_array_report[i][1], date_array_report[i][2],0,0,0)
        jd_arrival = jd + time_of_flight_report[i] / 24 / 3600
        r_1, r_2 = PosVecs(jd, jd_arrival)
        v_Mars, v_Earth = VelVecs(jd, jd_arrival)
        v_1, v_2, v_1_mag, v_2_mag = LambertIzzo(r_1, r_2, mu_Sun, time_of_flight_report[i])
        print("Date is " + str(date_array_report[i]) + " and time of flight is " + str(time_of_flight_report[i]))
        print("The magnitude of the outgong heliocentric velocity from Mars is " + str(v_1_mag / 1000))
        print("The magnitude of the incoming heliocentric velocity at Earth is " + str(v_2_mag / 1000))
        print("Hyperbolic excess velocity at Mars is " + str(magnitude(v_1 - v_Mars) / 1000))
        print("Hyperbolic excess velocity at Earth is " + str(magnitude(v_2 - v_Earth) / 1000))
        delta_v_1, delta_v_2 = delta_V_Mars_to_Earth_corrected(jd, jd_arrival, v_1, v_2, 3)
        print("The total $\Delta V$ necessary for the trip is " + str(delta_v_1 + delta_v_2))

    return "OK"

phi_range = np.deg2rad(np.linspace(-180, 180, 360001))



1.7453292520119845e-05
